In [1]:
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
import os
import vectorbt as vbt
import io
import sys
from contextlib import redirect_stdout
from datetime import datetime
import math
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import warnings
warnings.filterwarnings('ignore')

# Add project root to path
project_root = Path().absolute().parent
sys.path.append(str(project_root))

# Import utils with different aliases
from src.utils import csv_exporter as csv_utils
from src.utils import validation as val_utils
from src.utils import transformations as trans_utils
from src.utils import data_merger as merge_utils
from src.utils import config_validator as config_utils
from src.utils import metrics as metric_utils
from src.core.bloomberg_fetcher import fetch_bloomberg_data
from src.utils.transformations import get_ohlc



In [7]:
# Getting all the data 
mapping = {
    ('I05510CA Index', 'INDEX_OAS_TSY_BP'): 'cad_oas',
    ('LF98TRUU Index', 'INDEX_OAS_TSY_BP'): 'us_hy_oas',
    ('LUACTRUU Index', 'INDEX_OAS_TSY_BP'): 'us_ig_oas',
    ('SPTSX Index', 'PX_LAST'): 'tsx',
    ('VIX Index', 'PX_LAST'): 'vix',
}

# Calculate dates
end_date = datetime.now().strftime('%Y-%m-%d')
start_date ='2002-01-01'

# Fetch the data
df = fetch_bloomberg_data(
    mapping=mapping,
    start_date=start_date,
    end_date=end_date,
    periodicity='D',
    align_start=True
).dropna()

# Getting all the er_ytd data 
mapping1 = {
    ('I05510CA Index', 'INDEX_EXCESS_RETURN_YTD'): 'cad_ig_er',
    ('LF98TRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_hy_er',
    ('LUACTRUU Index', 'INDEX_EXCESS_RETURN_YTD'): 'us_ig_er',
}

# Fetch the er_ytd_data
df1 = fetch_bloomberg_data(
    mapping=mapping1,
    start_date=start_date,
    end_date=end_date,
    periodicity='D',
    align_start=True
).dropna()

# Convert er_ytd data to an index
df2 = trans_utils.convert_er_ytd_to_index(df1[['cad_ig_er','us_hy_er','us_ig_er']])
final_df = merge_utils.merge_dfs(df, df2, fill='ffill', start_date_align='yes')

# Handle bad data point for cad_oas on Nov 15 2005
bad_date = '2005-11-15'
if bad_date in final_df.index:
    final_df.loc[bad_date, 'cad_oas'] = final_df.loc[final_df.index < bad_date, 'cad_oas'].iloc[-1]

def calculate_qtd_changes(df):
    df_copy = df.copy()
    df_copy.index = pd.to_datetime(df_copy.index)
    
    qtd_changes = pd.DataFrame(index=df_copy.index)
    for col in df_copy.columns:
        changes = []
        for date in df_copy.index:
            # Get the start of the current quarter
            current_quarter_start = date.to_period('Q').start_time
            
            # Get the end of previous quarter (last day of previous quarter)
            prev_quarter_end = current_quarter_start - pd.Timedelta(days=1)
            
            # Get the value at the end of previous quarter
            mask = df_copy.index <= prev_quarter_end
            if not mask.any():
                changes.append(np.nan)
                continue
            
            prev_quarter_value = df_copy[col][mask].iloc[-1]
            current_value = df_copy[col][date]
            
            # Calculate change
            pct_change = (current_value / prev_quarter_value - 1) * 100
            changes.append(pct_change)
            
        qtd_changes[f'{col}_qtd'] = changes
    return qtd_changes

def calculate_ytd_changes(df):
    df_copy = df.copy()
    df_copy.index = pd.to_datetime(df_copy.index)
    
    ytd_changes = pd.DataFrame(index=df_copy.index)
    for col in df_copy.columns:
        changes = []
        for date in df_copy.index:
            # Get the start of the current year
            current_year_start = date.to_period('Y').start_time
            
            # Get the end of previous year (last day of previous year)
            prev_year_end = current_year_start - pd.Timedelta(days=1)
            
            # Get the value at the end of previous year
            mask = df_copy.index <= prev_year_end
            if not mask.any():
                changes.append(np.nan)
                continue
            
            prev_year_value = df_copy[col][mask].iloc[-1]
            current_value = df_copy[col][date]
            
            # Calculate change
            pct_change = (current_value / prev_year_value - 1) * 100
            changes.append(pct_change)
            
        ytd_changes[f'{col}_ytd'] = changes
    return ytd_changes

def add_all_changes(df):
    # Make a copy to avoid modifying original
    result_df = df.copy()
    result_df.index = pd.to_datetime(result_df.index)
    
    # Calculate regular percentage changes (3M and 1YR)
    for col in df.columns:
        # 3-month change (63 trading days approximation)
        result_df[f'{col}_3m'] = df[col].pct_change(periods=63) * 100
        # 1-year change (252 trading days approximation)
        result_df[f'{col}_1yr'] = df[col].pct_change(periods=252) * 100
    
    # Add QTD and YTD changes
    qtd_df = calculate_qtd_changes(df)
    ytd_df = calculate_ytd_changes(df)
    
    # Combine all changes
    for col in qtd_df.columns:
        result_df[col] = qtd_df[col]
    for col in ytd_df.columns:
        result_df[col] = ytd_df[col]
    
    # Drop any rows with missing values
    result_df = result_df.dropna()
    
    return result_df

def verify_calculations(df, n_random_samples=5):
    """Comprehensive verification of calculations"""
    print("Running verification checks...")
    print("\n1. Random Date Samples:")
    print("=" * 80)
    
    # Get random dates
    random_dates = df.sample(n=n_random_samples).index.sort_values()
    
    for date in random_dates:
        date = pd.to_datetime(date)
        
        # Get period boundaries
        current_quarter_start = date.to_period('Q').start_time
        current_year_start = date.to_period('Y').start_time
        prev_quarter_end = current_quarter_start - pd.Timedelta(days=1)
        prev_year_end = current_year_start - pd.Timedelta(days=1)
        
        print(f"\nDate: {date.strftime('%Y-%m-%d')}")
        print(f"Previous Quarter End: {prev_quarter_end.strftime('%Y-%m-%d')}")
        print(f"Previous Year End: {prev_year_end.strftime('%Y-%m-%d')}")
        
        # Check calculations for TSX
        col = 'tsx'
        current_value = df[col][date]
        prev_quarter_value = df[df.index <= prev_quarter_end][col].iloc[-1]
        prev_year_value = df[df.index <= prev_year_end][col].iloc[-1]
        
        manual_qtd = (current_value / prev_quarter_value - 1) * 100
        manual_ytd = (current_value / prev_year_value - 1) * 100
        
        print(f"\nTSX Values:")
        print(f"Current: {current_value:.2f}")
        print(f"Prev Quarter End: {prev_quarter_value:.2f}")
        print(f"Prev Year End: {prev_year_value:.2f}")
        print(f"QTD% (Manual): {manual_qtd:.2f}%")
        print(f"QTD% (Calculated): {df[f'{col}_qtd'][date]:.2f}%")
        print(f"YTD% (Manual): {manual_ytd:.2f}%")
        print(f"YTD% (Calculated): {df[f'{col}_ytd'][date]:.2f}%")
        
        # Check for significant differences
        qtd_diff = abs(manual_qtd - df[f'{col}_qtd'][date])
        ytd_diff = abs(manual_ytd - df[f'{col}_ytd'][date])
        
        if qtd_diff > 0.01:
            print(f"WARNING: Large QTD difference ({qtd_diff:.4f}%)")
        if ytd_diff > 0.01:
            print(f"WARNING: Large YTD difference ({ytd_diff:.4f}%)")
            
    print("\n2. Quarter/Year Boundary Checks:")
    print("=" * 80)
    
    # Find dates near quarter/year boundaries
    quarter_starts = df.index[pd.to_datetime(df.index).is_quarter_start]
    year_starts = df.index[pd.to_datetime(df.index).is_year_start]
    
    if len(quarter_starts) > 0:
        print("\nFirst day of quarter example:")
        first_quarter_date = quarter_starts[len(quarter_starts)//2]  # Take a middle example
        print(f"Date: {first_quarter_date}")
        print(f"TSX QTD%: {df.loc[first_quarter_date, 'tsx_qtd']:.2f}%")
        
    if len(year_starts) > 0:
        print("\nFirst day of year example:")
        first_year_date = year_starts[len(year_starts)//2]  # Take a middle example
        print(f"Date: {first_year_date}")
        print(f"TSX YTD%: {df.loc[first_year_date, 'tsx_ytd']:.2f}%")

# Calculate all changes
print("Calculating changes...")
final_df = add_all_changes(final_df)

# Run comprehensive verification
print("\nRunning verification...")
verify_calculations(final_df)

# Show basic info about the final dataset
print("\nFinal Dataset Info:")
print("=" * 80)
final_df.info()

Calculating changes...

Running verification...
Running verification checks...

1. Random Date Samples:

Date: 2010-09-03
Previous Quarter End: 2010-06-30
Previous Year End: 2009-12-31

TSX Values:
Current: 12144.92
Prev Quarter End: 11294.42
Prev Year End: 11746.11
QTD% (Manual): 7.53%
QTD% (Calculated): 7.53%
YTD% (Manual): 3.40%
YTD% (Calculated): 3.40%

Date: 2013-11-20
Previous Quarter End: 2013-09-30
Previous Year End: 2012-12-31

TSX Values:
Current: 13430.01
Prev Quarter End: 12787.19
Prev Year End: 12433.53
QTD% (Manual): 5.03%
QTD% (Calculated): 5.03%
YTD% (Manual): 8.01%
YTD% (Calculated): 8.01%

Date: 2016-07-13
Previous Quarter End: 2016-06-30
Previous Year End: 2015-12-31

TSX Values:
Current: 14493.80
Prev Quarter End: 14064.54
Prev Year End: 13009.95
QTD% (Manual): 3.05%
QTD% (Calculated): 3.05%
YTD% (Manual): 11.41%
YTD% (Calculated): 11.41%

Date: 2017-02-08
Previous Quarter End: 2016-12-31
Previous Year End: 2016-12-31

TSX Values:
Current: 15554.04
Prev Quarter End:

In [11]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType

# Create signals based on CAD OAS QTD being negative
signals = final_df['cad_oas_qtd'] < 0

# Create portfolio for our strategy
strategy_pf = vbt.Portfolio.from_signals(
    final_df['cad_ig_er_index'],
    entries=signals,
    exits=~signals,
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent,
    accumulate=False
)

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)

# Compare performance metrics
print("\nStrategy Performance:")
print(strategy_pf.stats())
print("\nBuy & Hold Performance:")
print(bh_pf.stats())


Strategy Performance:
Start                               2003-09-24 00:00:00
End                                 2025-01-24 00:00:00
Period                               5488 days 00:00:00
Start Value                                    100000.0
End Value                                 162043.853315
Total Return [%]                              62.043853
Benchmark Return [%]                           31.94987
Max Gross Exposure [%]                            100.0
Total Fees Paid                                     0.0
Max Drawdown [%]                               1.967439
Max Drawdown Duration                 508 days 00:00:00
Total Trades                                        169
Total Closed Trades                                 169
Total Open Trades                                     0
Open Trade PnL                                      0.0
Win Rate [%]                                  59.171598
Best Trade [%]                                 7.069267
Worst Trade [%]          

In [12]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType

# Create signals based on CAD OAS YTD being negative
signals = final_df['cad_oas_ytd'] < 0

# Create portfolio for our strategy
strategy_pf = vbt.Portfolio.from_signals(
    final_df['cad_ig_er_index'],
    entries=signals,
    exits=~signals,
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent,
    accumulate=False
)

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)

# Compare performance metrics
print("\nStrategy Performance:")
print(strategy_pf.stats())
print("\nBuy & Hold Performance:")
print(bh_pf.stats())


Strategy Performance:
Start                                2003-09-24 00:00:00
End                                  2025-01-24 00:00:00
Period                                5488 days 00:00:00
Start Value                                     100000.0
End Value                                  154744.803811
Total Return [%]                               54.744804
Benchmark Return [%]                            31.94987
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                                1.839838
Max Drawdown Duration                  485 days 00:00:00
Total Trades                                          83
Total Closed Trades                                   83
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   68.674699
Best Trade [%]                                 16.083912
Worst Tr

In [13]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType

# Create signals based on both CAD OAS QTD and YTD being negative
signals = (final_df['cad_oas_qtd'] < 0) & (final_df['cad_oas_ytd'] < 0)

# Create portfolio for our strategy
strategy_pf = vbt.Portfolio.from_signals(
    final_df['cad_ig_er_index'],
    entries=signals,
    exits=~signals,
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent,
    accumulate=False
)

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)

# Compare performance metrics
print("\nStrategy Performance:")
print(strategy_pf.stats())
print("\nBuy & Hold Performance:")
print(bh_pf.stats())


Strategy Performance:
Start                                2003-09-24 00:00:00
End                                  2025-01-24 00:00:00
Period                                5488 days 00:00:00
Start Value                                     100000.0
End Value                                  146422.127425
Total Return [%]                               46.422127
Benchmark Return [%]                            31.94987
Max Gross Exposure [%]                             100.0
Total Fees Paid                                      0.0
Max Drawdown [%]                                1.839838
Max Drawdown Duration                  509 days 00:00:00
Total Trades                                         150
Total Closed Trades                                  150
Total Open Trades                                      0
Open Trade PnL                                       0.0
Win Rate [%]                                   57.333333
Best Trade [%]                                  7.069267
Worst Tr

In [14]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType

# Create signals based on either CAD OAS QTD or YTD being negative
signals = (final_df['cad_oas_qtd'] < 0) | (final_df['cad_oas_ytd'] < 0)

# Create portfolio for our strategy
strategy_pf = vbt.Portfolio.from_signals(
    final_df['cad_ig_er_index'],
    entries=signals,
    exits=~signals,
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent,
    accumulate=False
)

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)

# Compare performance metrics
print("\nStrategy Performance:")
print(strategy_pf.stats())
print("\nBuy & Hold Performance:")
print(bh_pf.stats())


Strategy Performance:
Start                         2003-09-24 00:00:00
End                           2025-01-24 00:00:00
Period                         5488 days 00:00:00
Start Value                              100000.0
End Value                           171254.473153
Total Return [%]                        71.254473
Benchmark Return [%]                     31.94987
Max Gross Exposure [%]                      100.0
Total Fees Paid                               0.0
Max Drawdown [%]                         1.839838
Max Drawdown Duration           405 days 00:00:00
Total Trades                                  102
Total Closed Trades                           102
Total Open Trades                               0
Open Trade PnL                                0.0
Win Rate [%]                            70.588235
Best Trade [%]                          16.083912
Worst Trade [%]                          -0.43424
Avg Winning Trade [%]                    0.811917
Avg Losing Trade [%]       

In [15]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType
import pandas as pd

# Define our indicators and their types
positive_indicators = ['cad_ig_er_index', 'us_hy_er_index', 'us_ig_er_index', 'tsx']
negative_indicators = ['cad_oas', 'us_hy_oas', 'us_ig_oas', 'vix']
intervals = ['3m', '1yr', 'qtd', 'ytd']

# Dictionary to store all portfolios
portfolios = {}

# Create strategies for positive indicators
for indicator in positive_indicators:
    for interval in intervals:
        col_name = f'{indicator}_{interval}'
        strategy_name = f'{indicator}_{interval}_pos'
        signals = final_df[col_name] > 0
        
        portfolios[strategy_name] = vbt.Portfolio.from_signals(
            final_df['cad_ig_er_index'],
            entries=signals,
            exits=~signals,
            freq='1D',
            init_cash=100000,
            size=1.0,
            size_type=SizeType.Percent,
            accumulate=False
        )

# Create strategies for negative indicators
for indicator in negative_indicators:
    for interval in intervals:
        col_name = f'{indicator}_{interval}'
        strategy_name = f'{indicator}_{interval}_neg'
        signals = final_df[col_name] < 0
        
        portfolios[strategy_name] = vbt.Portfolio.from_signals(
            final_df['cad_ig_er_index'],
            entries=signals,
            exits=~signals,
            freq='1D',
            init_cash=100000,
            size=1.0,
            size_type=SizeType.Percent,
            accumulate=False
        )

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)
portfolios['buy_and_hold'] = bh_pf

# Create a comparison DataFrame
stats_list = []
for name, pf in portfolios.items():
    stats = pf.stats()
    stats_list.append(pd.Series(stats, name=name))

comparison_df = pd.DataFrame(stats_list)

# Display results
print("\nStrategy Comparison:")
print(comparison_df)


Strategy Comparison:
                             Start        End    Period  Start Value  \
cad_ig_er_index_3m_pos  2003-09-24 2025-01-24 5488 days     100000.0   
cad_ig_er_index_1yr_pos 2003-09-24 2025-01-24 5488 days     100000.0   
cad_ig_er_index_qtd_pos 2003-09-24 2025-01-24 5488 days     100000.0   
cad_ig_er_index_ytd_pos 2003-09-24 2025-01-24 5488 days     100000.0   
us_hy_er_index_3m_pos   2003-09-24 2025-01-24 5488 days     100000.0   
us_hy_er_index_1yr_pos  2003-09-24 2025-01-24 5488 days     100000.0   
us_hy_er_index_qtd_pos  2003-09-24 2025-01-24 5488 days     100000.0   
us_hy_er_index_ytd_pos  2003-09-24 2025-01-24 5488 days     100000.0   
us_ig_er_index_3m_pos   2003-09-24 2025-01-24 5488 days     100000.0   
us_ig_er_index_1yr_pos  2003-09-24 2025-01-24 5488 days     100000.0   
us_ig_er_index_qtd_pos  2003-09-24 2025-01-24 5488 days     100000.0   
us_ig_er_index_ytd_pos  2003-09-24 2025-01-24 5488 days     100000.0   
tsx_3m_pos              2003-09-24 2025-01

In [22]:
import vectorbt as vbt
from vectorbt.portfolio.enums import SizeType
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms
import random

# Clear any existing DEAP types
if 'FitnessMax' in creator.__dict__:
    del creator.FitnessMax
if 'Individual' in creator.__dict__:
    del creator.Individual

# Define our indicators and their types
positive_indicators = ['cad_ig_er_index', 'us_hy_er_index', 'us_ig_er_index', 'tsx']
negative_indicators = ['cad_oas', 'us_hy_oas', 'us_ig_oas', 'vix']
intervals = ['3m', '1yr', 'qtd', 'ytd']

# Generate all individual signals
signal_dict = {}

# Generate positive signals
for indicator in positive_indicators:
    for interval in intervals:
        col_name = f'{indicator}_{interval}'
        signal_name = f'{indicator}_{interval}_pos'
        signal_dict[signal_name] = final_df[col_name] > 0

# Generate negative signals
for indicator in negative_indicators:
    for interval in intervals:
        col_name = f'{indicator}_{interval}'
        signal_name = f'{indicator}_{interval}_neg'
        signal_dict[signal_name] = final_df[col_name] < 0

# Convert signals to list for indexing
signal_names = list(signal_dict.keys())
signals = list(signal_dict.values())

# Dictionary to store all portfolios
all_portfolios = {}

# First evaluate individual signals
print("Evaluating individual signals...")
for name, signal in signal_dict.items():
    pf = vbt.Portfolio.from_signals(
        final_df['cad_ig_er_index'],
        entries=signal,
        exits=~signal,
        freq='1D',
        init_cash=100000,
        size=1.0,
        size_type=SizeType.Percent,
        accumulate=False
    )
    all_portfolios[name] = pf

# Create comparison DataFrame for individual strategies
individual_results = []
for name, pf in all_portfolios.items():
    stats = pf.stats()
    individual_results.append({
        'Strategy': name,
        'Total Return [%]': stats['Total Return [%]'],
        'Sharpe Ratio': stats['Sharpe Ratio'],
        'Max Drawdown [%]': stats['Max Drawdown [%]'],
        'Win Rate [%]': stats['Win Rate [%]']
    })

print("\nTop 5 Individual Strategies:")
individual_df = pd.DataFrame(individual_results).sort_values('Total Return [%]', ascending=False)
print(individual_df.head())

# Genetic Algorithm Setup
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

def evaluate(individual):
    used_signals = []
    operations = []
    combined_signal = None
    
    for i in range(len(signals)):
        if individual[i * 2]:  # If signal is used
            signal = signals[i]
            signal_name = signal_names[i]
            
            if signal_name in used_signals:
                continue
                
            used_signals.append(signal_name)
            
            if combined_signal is None:
                combined_signal = signal
            else:
                op_idx = i * 2 - 1
                if op_idx < len(individual):
                    operation = "OR" if individual[op_idx] else "AND"
                    operations.append(operation)
                    
                    if operation == "OR":
                        combined_signal = combined_signal | signal
                    else:
                        combined_signal = combined_signal & signal
    
    if combined_signal is None or len(used_signals) < 2:
        return -np.inf,
    
    pf = vbt.Portfolio.from_signals(
        final_df['cad_ig_er_index'],
        entries=combined_signal,
        exits=~combined_signal,
        freq='1D',
        init_cash=100000,
        size=1.0,
        size_type=SizeType.Percent,
        accumulate=False
    )
    
    # Store portfolio with strategy description
    strategy_key = "Combined:" + ":".join(used_signals)
    operations_str = ":".join(operations)
    all_portfolios[f"{strategy_key}:{operations_str}"] = pf
    
    return pf.stats()['Total Return [%]'],

def create_diverse_individual():
    ind = [0] * n_bits
    n_signals_to_use = random.randint(2, 4)
    signal_positions = random.sample(range(n_signals), n_signals_to_use)
    
    for pos in signal_positions:
        ind[pos * 2] = 1
        if pos > 0:
            ind[pos * 2 - 1] = random.randint(0, 1)
    
    return creator.Individual(ind)

# Initialize genetic algorithm
toolbox = base.Toolbox()
n_signals = len(signals)
n_bits = 2 * n_signals - 1

toolbox.register("individual", create_diverse_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.1)
toolbox.register("select", tools.selNSGA2)

# Genetic Algorithm Parameters
POPULATION_SIZE = 200
MAX_GENERATIONS = 100
HALL_OF_FAME_SIZE = 10

# Create hall of fame
hall_of_fame = tools.HallOfFame(HALL_OF_FAME_SIZE)

# Run genetic algorithm
population = toolbox.population(n=POPULATION_SIZE)
algorithms.eaSimple(population, toolbox, 
                   cxpb=0.8,
                   mutpb=0.3,
                   ngen=MAX_GENERATIONS, 
                   halloffame=hall_of_fame,
                   verbose=True)

# Create results DataFrame for all strategies
all_results = []
for name, pf in all_portfolios.items():
    stats = pf.stats()
    ret_stats = pf.returns.stats()
    
    result = {
        'Strategy': name,
        'Total Return [%]': stats['Total Return [%]'],
        'Sharpe Ratio': stats['Sharpe Ratio'],
        'Max Drawdown [%]': stats['Max Drawdown [%]'],
        'Win Rate [%]': stats['Win Rate [%]'],
        'Profit Factor': stats['Profit Factor'],
        'Expectancy': stats['Expectancy'],
        'Sortino Ratio': stats['Sortino Ratio'],
        'Calmar Ratio': stats['Calmar Ratio']
    }
    all_results.append(result)

results_df = pd.DataFrame(all_results)
results_df = results_df.sort_values('Total Return [%]', ascending=False)

print("\nTop 10 Strategies Overall:")
print(results_df.head(10))

# Get best strategy
best_strategy_name = results_df.iloc[0]['Strategy']
best_portfolio = all_portfolios[best_strategy_name]

# Create buy & hold portfolio for comparison
bh_pf = vbt.Portfolio.from_holding(
    final_df['cad_ig_er_index'],
    freq='1D',
    init_cash=100000,
    size=1.0,
    size_type=SizeType.Percent
)

print("\nBest Strategy:", best_strategy_name)
print("\nBest Strategy Performance:")
print(best_portfolio.stats())
print("\nBuy & Hold Performance:")
print(bh_pf.stats())

# Additional analysis of returns
print("\nBest Strategy Returns Analysis:")
print(best_portfolio.returns.stats())
print("\nBuy & Hold Returns Analysis:")
print(bh_pf.returns.stats())

Evaluating individual signals...

Top 5 Individual Strategies:
                  Strategy  Total Return [%]  Sharpe Ratio  Max Drawdown [%]  \
6   us_hy_er_index_qtd_pos         83.932896      3.366819          1.839838   
26       us_ig_oas_qtd_neg         83.233926      3.446509          1.839838   
10  us_ig_er_index_qtd_pos         81.397454      3.395472          1.839838   
14             tsx_qtd_pos         80.090655      3.449959          2.194076   
22       us_hy_oas_qtd_neg         77.869232      3.319215          1.712487   

    Win Rate [%]  
6      65.486726  
26     76.041667  
10     72.222222  
14     72.352941  
22     68.571429  
gen	nevals
0  	200   
1  	184   
2  	181   
3  	165   
4  	173   
5  	179   
6  	169   
7  	177   
8  	174   
9  	164   
10 	171   
11 	178   
12 	171   
13 	178   
14 	171   
15 	167   
16 	164   
17 	167   
18 	166   
19 	176   
20 	166   
21 	172   
22 	177   
23 	174   
24 	181   
25 	180   
26 	168   
27 	174   
28 	165   
29 	168   
3

AttributeError: 'function' object has no attribute 'stats'